In [2]:
%cd ..

/home/mmansour/Music/AUB/Julia/TempRepo


In [3]:
from copy import deepcopy
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import warnings

import pandas as pd

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings("ignore")
from carla.data.api import data
import numpy as np
import torch
torch.cuda.is_available = lambda : False
import yaml
from seed_env import seed_my_session
from typing import Dict, List
from cent.data_specific import DataModels
from carla import Benchmark
import pandas as pd
from carla.recourse_methods import (
    CCHVAE,
    CEM,
    CRUD,
    FOCUS,
    CausalRecourse,
    Clue,
    Dice,
    Face,
    FeatureTweak,
    GrowingSpheres,
    Revise,
    Wachter,
)
from carla.recourse_methods.catalog.causal_recourse import constraints, samplers
import carla.evaluation.catalog as evaluation_catalog
from cent.method import CEnt
from vae_benchmark import VAEBenchmark
from tensorflow import Graph, Session
from carla.models.catalog import MLModelCatalog

Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]
[WARNING] From /tmp/ipykernel_88741/3118733197.py:40: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.
 [deprecation_wrapper.py __getattr__]


In [4]:
data_name = 'adult'
FACTUAL_NUMBER = 20
OUT_DIR_DATA = 'outputs/tmp/'
if not os.path.exists(OUT_DIR_DATA):
    os.makedirs(OUT_DIR_DATA)
data_models = DataModels(data_name = data_name,
                             factuals_length = FACTUAL_NUMBER,
                             out_dir = OUT_DIR_DATA)

Loading models... --- logs will be saved to outputs/tmp/models_logs.txt
[WARNING] From /home/mmansour/miniconda3/envs/xai/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor [deprecation.py new_func]
[WARNING] From /home/mmansour/miniconda3/envs/xai/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor [deprecation.py new_func]


In [20]:
layers = [25]
latent_dim = 7
mlmodel = data_models.models_zoo['ann']['tensorflow']
min_entries_per_label = int(data_models.trainData.df.shape[0]*0.01)
MIN_ENTRIES_PER_LABEL_THRESH = 500
if min_entries_per_label<MIN_ENTRIES_PER_LABEL_THRESH:
    print('min_entries_per_label is too small {}, setting it to {} '.format(min_entries_per_label,MIN_ENTRIES_PER_LABEL_THRESH))
#TODO: @MM Return this to 1% of data
min_entries_per_label = MIN_ENTRIES_PER_LABEL_THRESH
hpr = {"data_name":"adult",
        "myvae_params": {
            'input_dim': len(mlmodel.feature_input_order),
            "epochs":16
        },
        "tree_params": {
            "min_entries_per_label": min_entries_per_label,
            "max_search" : 50,
            "grid_search": {"cv": 1,"splitter": ["best"],"criterion": ["gini"],"max_depth": [3,4,5,6,7],
                            "min_samples_split": [1.0,2,3],"min_samples_leaf": [1,2,3],
                            "max_features": ['sqrt',1.0, 'log2',0.8],
                            }
        }
    }
cent_method =  CEnt(deepcopy(data_models.trainData), mlmodel, hpr, data_catalog= data_models.new_catalog_n)

min_entries_per_label is too small 487, setting it to 500 
{'input_dim': 13, 'kld_weight': 0.0025, 'layers': [32, 16], 'latent_dim': 7, 'hidden_activation': 'relu', 'dropout': 0.2, 'batch_norm': True, 'batch_size': 64, 'epochs': 16, 'learning_rate': 0.001, 'weight_decay': 0.0, 'cuda': False, 'verbose': True, 'train': True, 'save_dir': './vae_model/'}
./vae_model/adult
Epoch: 0, ELBO Loss: 104.75636291503906, Test MSELoss: 55.0160026550293
Epoch: 0, Best ELBO Loss: 104.75636291503906
Epoch: 1, ELBO Loss: 49.49557876586914, Test MSELoss: 45.0239372253418
BEST Epoch: 1, Best ELBO Loss: 49.49557876586914
Epoch: 2, ELBO Loss: 43.653743743896484, Test MSELoss: 41.34355163574219
BEST Epoch: 2, Best ELBO Loss: 43.653743743896484
Epoch: 3, ELBO Loss: 40.522613525390625, Test MSELoss: 38.45353698730469
BEST Epoch: 3, Best ELBO Loss: 40.522613525390625
Epoch: 4, ELBO Loss: 37.70673751831055, Test MSELoss: 35.75862503051758
BEST Epoch: 4, Best ELBO Loss: 37.70673751831055
Epoch: 5, ELBO Loss: 35.0

In [23]:
from time import time
fact = data_models.factuals['ann'].sample(1)

In [26]:
start_timer = time()
contrast = cent_method.get_counterfactuals(fact)
print('Time taken to get contrast: {:.2f} seconds'.format(time()-start_timer))
contrast

Time taken to get contrast: 0.02 seconds


,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,relationship_Non-Husband,race_White,sex_Male,native-country_US
0,0.164384,0.076916,0.764858,0.071366,0.0,...,1.0,1.0,1.0,0.0,1.0


In [35]:
cent_method.explain_in_text(fact)

To flip the binary class from 0 to 1, we moved our factual to a point satisfying the following conditions:
	education-num needs to be > than 0.7666666805744171
	marital-status_Non-Married needs to be flipped to False
	capital-gain needs to be > than 0.0748707503080368
